# Sources:
1 - https://karpathy.ai/zero-to-hero.html  
2 - https://www.youtube.com/watch?v=kCc8FmEb1nY  
3 - https://github.com/karpathy/minbpe/tree/master  

# Import libraries

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import time
import re

# Parameters

In [2]:
# Dataset parameters
train_ratio = 0.9
sample_size = 10

SEED = 15

# Load files

In [3]:
from datasets import load_dataset

# Define the path to the dataset
dataset_name = "20231101.fr"

# Load the dataset
raw_dataset = load_dataset("wikimedia/wikipedia", dataset_name)

c:\Users\hzf04d\Anaconda3\envs\llm2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|██████████| 17/17 [00:02<00:00,  7.29it/s]


# Data preparation

In [4]:
raw_dataset['train'].num_rows

2564646

In [5]:
# Create training and evaluation datasets
if sample_size < 0:
    train_sample = round(raw_dataset['train'].num_rows * train_ratio)
    test_sample = round(raw_dataset['train'].num_rows * (1- train_ratio))
else:
    train_sample = round(sample_size * train_ratio)
    test_sample = round(sample_size * (1- train_ratio))

ds_train_test = raw_dataset['train'].train_test_split(train_size=train_sample, test_size=test_sample, seed=SEED)
train_text = ''.join([t['text'] for t in ds_train_test['train']])

In [6]:
# # Create the vocabulary from the training data
# def create_vocab(text, type="char"):
#     if type == "char":
#         sorted_text = sorted(list(set(text)))
#     elif type == "word":
#         sorted_text = sorted(list(set(text)))
#     else:
#         raise Exception("Invalid type")
    
#     text_to_int = {c: i for i, c in enumerate(sorted_text)}
#     int_to_text = {i: c for i, c in enumerate(sorted_text)}

#     return text_to_int, int_to_text
    

# def char_encoder(text, text_to_int):
#     return [text_to_int[char] for char in text]


# def get_most_common_token_pair(tokens, verbose=False, int_to_text=None):
#     d = {}
#     for pairs in tqdm(zip(tokens, tokens[1:])):
#         d[pairs] = d.get(pairs, 0) + 1

#     # Get the key having the highest value (higher frequency of occurrence)
#     most_freq = max(d, key=lambda k: d[k])
#     if verbose:
#         print(most_freq)
#         most_freq_text = "".join([int_to_text[i] for i in most_freq])
#         print(f'Most frequent word: {most_freq} --> "{most_freq_text}"')
#     return most_freq

In [7]:
# max_vocab_size = 150
# text_to_int, int_to_text = create_vocab(train_text, "char")
# train_tokens = char_encoder(train_text, text_to_int)    
# train_tokens_temp = train_tokens.copy()
# vocab_size = len(text_to_int)
# print("Vocab size:", vocab_size, "Sequence length:", len(train_tokens_temp))

# while vocab_size < max_vocab_size:
#     # Get the most frequent token pair
#     most_freq = get_most_common_token_pair(train_tokens_temp, True, int_to_text)
    
#     # Update the vocabulary
#     token_pair = (most_freq[0], most_freq[1])
#     str_pair = ''.join([int_to_text[most_freq[0]], int_to_text[most_freq[1]]])
#     text_to_int[str_pair] = vocab_size
#     int_to_text[vocab_size] = str_pair

#     # Replace the most frequent token pair with a new one
#     i = 0
#     new_train_token = []
#     while i < len(train_tokens_temp) - 1:
#         if (train_tokens_temp[i], train_tokens_temp[i+1]) == token_pair:
#             new_train_token.append(text_to_int[str_pair])
#             i += 2
#         else:
#             new_train_token.append(train_tokens_temp[i])
#             i += 1
    
#     # Print the statistics
#     vocab_size = len(text_to_int)
#     print("Vocab size:", vocab_size, "Sequence length:", len(new_train_token))
#     train_tokens_temp = new_train_token
    


In [8]:
class BasicTokenizer():
    def __init__(self, ):
        pass
        
    def train(self, text, max_vocab_size, verbose=False):
        self.text_to_int, self.int_to_text = self._create_vocab(text)
        train_tokens = self._char_encoder(text, self.text_to_int)    
        train_tokens_temp = train_tokens.copy()
        current_vocab_size = len(self.text_to_int)
        
        if verbose: 
            print("Vocab size:", current_vocab_size, "Sequence length:", len(train_tokens_temp))
        
        while current_vocab_size < max_vocab_size:
            # Get the most frequent token pair
            most_freq = self._get_most_common_token_pair(train_tokens_temp, True, self.int_to_text)
            
            # Update the vocabulary
            token_pair = (most_freq[0], most_freq[1])
            str_pair = ''.join([self.int_to_text[most_freq[0]], self.int_to_text[most_freq[1]]])
            self.text_to_int[str_pair] = current_vocab_size
            self.int_to_text[current_vocab_size] = str_pair

            # Replace the most frequent token pair with a new one
            i = 0
            new_train_token = []
            while i < len(train_tokens_temp) - 1:
                if (train_tokens_temp[i], train_tokens_temp[i+1]) == token_pair:
                    new_train_token.append(self.text_to_int[str_pair])
                    i += 2
                else:
                    new_train_token.append(train_tokens_temp[i])
                    i += 1
            
            # Print the statistics
            if verbose:
                current_vocab_size = len(self.text_to_int)
                print("Vocab size:", current_vocab_size, "Sequence length:", len(new_train_token))
                
            train_tokens_temp = new_train_token
            
    # Create the vocabulary from the training data
    def _create_vocab(self, text):
        sorted_text = sorted(list(set(text)))
        text_to_int = {c: i for i, c in enumerate(sorted_text)}
        int_to_text = {i: c for i, c in enumerate(sorted_text)}
        return text_to_int, int_to_text
        
    def _char_encoder(self, text, text_to_int):
        return [text_to_int[char] for char in text]

    def _get_most_common_token_pair(self, tokens, verbose=False, int_to_text=None):
        d = {}
        for pairs in tqdm(zip(tokens, tokens[1:])):
            d[pairs] = d.get(pairs, 0) + 1

        # Get the key having the highest value (higher frequency of occurrence)
        most_freq = max(d, key=lambda k: d[k])
        if verbose:
            print(most_freq)
            most_freq_text = "".join([int_to_text[i] for i in most_freq])
            print(f'Most frequent word: {most_freq} --> "{most_freq_text}"')
        return most_freq
    
    def encode(self, text):
        return self._char_encoder(text, self.text_to_int)
    
    def decode(self, tokens):
        return "".join([self.int_to_text[i] for i in tokens])

In [9]:
tokenizer = BasicTokenizer()
tokenizer.train(train_text, max_vocab_size=200, verbose=True)

Vocab size: 94 Sequence length: 35611


35610it [00:00, 2099923.59it/s]


(54, 2)
Most frequent word: (54, 2) --> "e "
Vocab size: 95 Sequence length: 34075


34074it [00:00, 1798305.27it/s]


(68, 2)
Most frequent word: (68, 2) --> "s "
Vocab size: 96 Sequence length: 33267


...

(55, 55)
Most frequent word: (55, 55) --> "ff"
Vocab size: 198 Sequence length: 20287


20286it [00:00, 1719874.90it/s]


(54, 56)
Most frequent word: (54, 56) --> "eg"
Vocab size: 199 Sequence length: 20244


20243it [00:00, 1803502.61it/s]

(191, 181)
Most frequent word: (191, 181) --> "province de "
Vocab size: 200 Sequence length: 20201


In [10]:
tokenizer.encode('Bonjour')

[25, 64, 63, 59, 64, 70, 67]

In [11]:
tokenizer.decode([25, 64, 63, 59, 64, 70, 67])

'Bonjour'